In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

num_samples = 100 # data sample count
time_steps = 200  # Number of time steps in each sample
num_sensors = 6   # Number of sensors

# Define the full file paths for your files
data_files = [
    'sensor1_data.txt',
    'sensor2_data.txt',
    'sensor3_data.txt',
    'sensor4_data.txt',
    'sensor5_data.txt',
    'sensor6_data.txt'
]

label_file = "label_data.txt"

In [2]:
# Initialize an empty list to store data for each channel
input_data = []
input_labels = []

# Load data from each text file and append it to channel_data
for filename in data_files:
    with open(filename, 'r') as file:
        lines = file.readlines()
        channel_values = [list(map(float, line.strip().split())) for line in lines]
        input_data.append(channel_values)

x = np.array(input_data).transpose(1, 2, 0)

# Load labels from the label file
with open(label_file, 'r') as file:
    lines = file.readlines()
    input_labels = [int(value) for line in lines for value in line.strip().split()]

y_labels = np.array(input_labels)

y = to_categorical(y_labels-1,num_classes=2)  # One-hot encode the numerical labels

In [3]:
# check the data in right shape
print(x.shape)
print(y_labels.shape)

(100, 200, 6)
(100,)


In [4]:
# Split the data into training and test sets
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
# Building the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_sensors)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 2 different people

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
# Training the model
model.fit(x, y, epochs=20, batch_size=2, validation_split=0.2)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9472 - loss: 0.1512 - val_accuracy: 0.8000 - val_loss: 0.8713
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8942 - loss: 0.2405 - val_accuracy: 0.2500 - val_loss: 2.7443
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8871 - loss: 0.6967 - val_accuracy: 0.4500 - val_loss: 1.6225
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9382 - loss: 0.4168 - val_accuracy: 0.6500 - val_loss: 0.8332
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8751 - loss: 0.4362 - val_accuracy: 0.4000 - val_loss: 1.5280
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8594 - loss: 0.3127 - val_accuracy: 0.6000 - val_loss: 0.9838
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8396 - loss: 0.8464 - val_accuracy: 0.1500 - val_loss: 3.1985
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9352 - loss: 0.3700 - val_accuracy: 0.4500 - val_loss:

In [8]:
model.save('trained_model.h5') # Save the trained model

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.utils import to_categorical

model = load_model('trained_model.h5') # Load the trained model

In [4]:
# Define the full file paths for your files
test_data_files = [
    'sensor1_test.txt',
    'sensor2_test.txt',
    'sensor3_test.txt',
    'sensor4_test.txt',
    'sensor5_test.txt',
    'sensor6_test.txt'
]

test_label_file = "label_test.txt"

# Initialize an empty list to store data for each channel
input_test_data = []
input_test_labels = []

# Load data from each text file and append it to channel_data
for filename in test_data_files:
    with open(filename, 'r') as file:
        lines = file.readlines()
        channel_values = [list(map(float, line.strip().split())) for line in lines]
        input_test_data.append(channel_values)

x_test = np.array(input_test_data).transpose(1, 2, 0)

# Load labels from the label file
with open(test_label_file, 'r') as file:
    lines = file.readlines()
    input_test_labels = [int(value) for line in lines for value in line.strip().split()]

y_test_labels = np.array(input_test_labels)

y_test = to_categorical(y_test_labels-1,num_classes=2)  # One-hot encode the numerical labels

# print(x_test)
# print(y_test)

In [5]:
loss, accuracy = model.evaluate(x_test, y_test)
y_pred = model.predict(x_test)
y_pred_binary = np.argmax(y_pred, axis=1)
y_true_binary = np.argmax(y_test, axis=1)
print(y_true_binary)
print(y_pred_binary)

print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.6538 - loss: 1.1914
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0]
Test Loss: 1.1914, Test Accuracy: 0.6538


In [20]:
a = "1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1"
b = "2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2"

l = b.strip().split()
print(len(l))

63
